In [3]:
from utils.general import *
from utils.video_processing import *
from utils.chat_api import *
from prompts import *
import tempfile

from moviepy import VideoFileClip
import base64
import os
import numpy as np
from tqdm import tqdm

In [5]:
def answer_question(video_clips, qa, model):
    question = qa["question"]
    gt = qa["answer"]

    inputs = [
        [
            {"type": "video_base64", "content": clip},
            {"type": "text", "content": prompt_baseline_answer_clipwise_extract},
            {"type": "text", "content": f"Question: {question}"},
            {"type": "text", "content": "Extracted information:"},
        ]
        for clip in video_clips
    ]
    messages = [generate_messages(input) for input in inputs]
    responses = parallel_get_response(model, messages)

    extracted_information = [
        response for response in responses[0] if not response.lower().startswith("none")
    ]
    qa["extracted_information"] = extracted_information
    if len(extracted_information) == 0:
        answer = "Unanswerable."
    else:
        input = [
            {"type": "text", "content": prompt_baseline_answer_clipwise_summarize},
            {"type": "text", "content": f"Question: {question}"},
            {
                "type": "text",
                "content": f"Extracted information: {extracted_information}",
            },
            {"type": "text", "content": "Answer:"},
        ]
        messages = generate_messages(input)
        model = "gpt-4o-2024-11-20"
        response = get_response_with_retry(model, messages)
        answer = response[0]

    qa["answer_baselines"] = answer
    return qa


def verify_answers(qas):
    inputs = [
        [
            {
                "type": "text",
                "content": json.dumps(qa),
            },
            {
                "type": "text",
                "content": prompt_benchmark_verify_answer,
            },
            {
                "type": "text",
                "content": "Now answer if the answer from the baseline is correct or not:",
            },
        ]
        for qa in qas
    ]
    messages = [generate_messages(input) for input in inputs]
    model = "gpt-4o-2024-11-20"
    responses = parallel_get_response(model, messages)

    results = responses[0]

    # calculate the accuracy of the answers
    correct = 0
    for result in results:
        if result.lower().startswith("yes"):
            correct += 1
    accuracy = correct / len(results)

    return accuracy, results


def process_video(video, interval_seconds):
    """Process video segments at specified intervals with given fps.

    Args:
        video_path (str): Path to the video file
        interval_seconds (float): Time interval between segments in seconds
        fps (float): Frames per second to extract from each segment

    Returns:
        None
    """

    video_path = video["path"]
    qa_list = video["qa_list"]

    video_info = get_video_info(video_path)
    print(video_info)

    clips = []

    # Process each interval
    for start_time in np.arange(0, video_info["duration"], interval_seconds):
        if start_time >= video_info["duration"]:
            break

        base64_video, _, _ = process_video_clip(
            video_path, start_time, interval_seconds
        )
        clips.append(base64_video)

    model = "gemini-1.5-pro-002"
    qpm = config[model]["qpm"]
    qa_batch_size = qpm // len(clips)
    qa_batches = [
        qa_list[i : i + qa_batch_size] for i in range(0, len(qa_list), qa_batch_size)
    ]

    answered_qa_list = []

    for qa_batch in qa_batches:
        # parallel question answering with multiple threads
        with ThreadPoolExecutor(max_workers=len(qa_batch)) as executor:
            futures = [
                executor.submit(answer_question, clips, qa, model) for qa in qa_batch
            ]
            for future in as_completed(futures):
                qa = future.result()
                answered_qa_list.append(qa)

    video["qa_list"] = answered_qa_list
    return video

In [5]:
import json

with open("data/annotations/video_list_CZ_modified.json", "r") as f:
    data = json.load(f)

for video in tqdm(data):
    # print(video)
    video = process_video(video, 180)
    with open("data/annotations/video_list_CZ_answer_clipwise_0320.json", "w") as f:
        json.dump(data, f, indent=4)

  0%|          | 0/50 [00:00<?, ?it/s]

{'fps': 23.976023976023978, 'frames': 41829, 'duration': 1744.617875, 'path': 'data/videos/raw/360p/PnvZZwlN2yk.mp4', 'name': 'PnvZZwlN2yk.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}
{'fps': 23.976023976023978, 'frames': 41829, 'duration': 1744.617875, 'path': 'data/videos/raw/360p/PnvZZwlN2yk.mp4', 'name': 'PnvZZwlN2yk.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}


  2%|▏         | 1/50 [03:54<3:11:42, 234.75s/it]

{'fps': 23.976023976023978, 'frames': 39306, 'duration': 1639.3877499999999, 'path': 'data/videos/raw/360p/n-B_kmAebbQ.mp4', 'name': 'n-B_kmAebbQ.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}
{'fps': 23.976023976023978, 'frames': 39306, 'duration': 1639.3877499999999, 'path': 'data/videos/raw/360p/n-B_kmAebbQ.mp4', 'name': 'n-B_kmAebbQ.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}


  4%|▍         | 2/50 [08:25<3:24:56, 256.19s/it]

...

 96%|█████████▌| 48/50 [3:51:08<09:19, 279.94s/it]

{'fps': 25.0, 'frames': 36424, 'duration': 1456.96, 'path': 'data/videos/raw/360p/oTzSzfZGV2E.mp4', 'name': 'oTzSzfZGV2E.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}
{'fps': 25.0, 'frames': 36424, 'duration': 1456.96, 'path': 'data/videos/raw/360p/oTzSzfZGV2E.mp4', 'name': 'oTzSzfZGV2E.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}


 98%|█████████▊| 49/50 [3:55:54<04:41, 281.68s/it]

{'fps': 23.976024029573885, 'frames': 29819, 'duration': 1243.700788888889, 'path': 'data/videos/raw/360p/DNu4WiQaVTI.mp4', 'name': 'DNu4WiQaVTI.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}
{'fps': 23.976024029573885, 'frames': 29819, 'duration': 1243.700788888889, 'path': 'data/videos/raw/360p/DNu4WiQaVTI.mp4', 'name': 'DNu4WiQaVTI.mp4', 'width': 640, 'height': 360, 'codec': 808596553.0, 'format': 0.0, 'fourcc': 875967080.0}


100%|██████████| 50/50 [3:59:14<00:00, 287.10s/it]


In [6]:
with open("data/annotations/video_list_CZ_answer_clipwise_0320.json", "r") as f:
    video_list = json.load(f)

qas = []
for video in video_list:
    qa_list = [
        {
            "question": qa["question"],
            "ground truth answer": qa["answer"],
            "baseline answer": qa["answer_baselines"],
        }
        for qa in video["qa_list"]
    ]
    qas.extend(qa_list)

acc, results = verify_answers(qas)

print(acc)

idx = 0
for video in video_list:
    for qa in video["qa_list"]:
        qa["correct"] = results[idx]
        idx += 1

with open(
    "data/annotations/video_list_CZ_answer_clipwise_verified_0320.json", "w"
) as f:
    json.dump(video_list, f, indent=4)

Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'mess

In [5]:
print(results)

['Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No',